# setup

In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import trimesh
import cloudvolume as cv
from google.cloud import bigquery
from cloudvolume import CloudVolume 
import os
from sqlite3 import connect as sqlite3_connect
from sqlite3 import DatabaseError
from cloudvolume.exceptions import EmptyFileException, MeshDecodeError
# from google.oauth2 import service_account

In [2]:
import sys
sys.path.append('/Users/kperks/Documents/ell-connectome/efish_em/efish_em')

In [3]:
# from eCREST_cli_beta import ecrest, import_settings
# from eCREST_cli import ecrest
import AnalysisCode as efish 

In [4]:
'''
conda install conda-forge::google-cloud-sdk

Then, launch jupyter lab 

In a code cell, run bash command <!gcloud auth login > (https://cloud.google.com/sdk/gcloud/reference/auth/login)
    a browser tab should open up

RESULT:
You are now logged in as [kperky@gmail.com].
Your current project is [lcht-goog-connectomics].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
'''
!gcloud auth login 

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=wi9nngNJu9A8O8dDXDZAp6Zx0knqXc&access_type=offline&code_challenge=h2K03dDs9OH_2Maf8pRfz5GNVRuO3m0___l764--pBI&code_challenge_method=S256


You are now logged in as [kperky@gmail.com].
Your current project is [lcht-goog-connectomics].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [5]:
import sys
sys.path.append('/Users/kperks/Documents/ell-connectome/efish_em/efish_em')

In [6]:
# from eCREST_cli_beta import ecrest, import_settings
from eCREST_cli import ecrest, import_settings, get_cell_filepaths

In [7]:
path_to_settings_json = '/Users/kperks/Documents/ell-connectome/eCREST-local-files/settings_dict.json'
settings_dict = import_settings(path_to_settings_json)

vx_sizes = [16,16,30]

db_cursors = sqlite3_connect(settings_dict['db_path'], check_same_thread=False).cursor()

a = ', '.join(['base_address'])

db_cursors.execute(f'''SELECT {a} FROM addresses_table LIMIT 1''')

[base_seg] = db_cursors.fetchall()[0]

mesh_seg = 'precomputed://gs+ngauth+https://ngauth-goog.appspot.com/fish-ell/roi450um_seg32fb16fb_220930'

In [8]:
efish_cloudvolume = cv.CloudVolume('gs://fish-ell/roi450um_seg32fb16fb_220930', progress=False)
# efish_cloudvolume = cv.CloudVolume('precomputed://http://localhost:8001', progress=False)


In [9]:
# efish_img = cv.CloudVolume('gs://fish-ell/roi450um_xyz', progress=False)

# load file list

In [10]:
dirpath = Path(settings_dict['save_dir'])

nodefiles = get_cell_filepaths(dirpath)

In [11]:
df_type = pd.read_csv(dirpath / 'metadata/df_type_auto_typed.csv')

In [13]:
# resort existing and figure out which needed

# export cells

In [32]:
savepath = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/ms/figures_final/blender/obj/Fig4/mg1-input_mg2-387338914')

In [39]:
savepath

PosixPath('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/ms/figures_final/blender/obj/Fig4/convergence_lg214503199/mg2_cell_parts')

In [36]:
all_cells =  [128536704] # df_type[df_type['cell_type'].isin(['mg1','mg2'])]['id'].unique()#sg

       # df_type[df_type['cell_type'].isin(['mg1','mg2'])]['id'].unique() # mg

In [38]:
howmuch = 'single' #'list' #'all'  #
dtype = 'axon'
dtype_list = ['unknown','basal dendrite','apical dendrite','multiple']
reduction_ratio = 0.2

scale_factor = 100000
yaxis_replace_scale = 32768*16/scale_factor
# zaxis_replace_scale = 3534*30/scale_factor
refpt = [0,0,0]

# with tqdm(total=len(all_cells)) as pbar:        
for focal_cell_id in all_cells:
    # pbar.update(1)
    
    dirpath = Path(settings_dict['save_dir'])
    focal_cell_fname = nodefiles[str(focal_cell_id)]#

    cell_data = efish.load_ecrest_celldata(nodefiles[str(focal_cell_id)])
    # cell = ecrest(settings_dict,filepath = dirpath / focal_cell_fname, launch_viewer=False)
    ctype = df_type[df_type['id']==focal_cell_id]['cell_type'].values[0]#cell.get_ctype('manual')

    if howmuch == 'list':
        mesh_ids = [bs for dtype in dtype_list for bs in cell_data['base_segments'][dtype]] # for subset list of cell structures
    if howmuch == 'single':
        mesh_ids = [bs for bs in cell_data['base_segments'][dtype]] # if only need segments from one cell structure #
    if howmuch == 'all':
        mesh_ids = [bs for dtype in cell_data['base_segments'].keys() for bs in cell_data['base_segments'][dtype]] #if need all segments

    # grab the mesh for more than one object
    # Initialize an empty Trimesh object
    fused_mesh = trimesh.Trimesh()
    with tqdm(total=len(mesh_ids)) as pbar:        
        for id in mesh_ids:
            pbar.update(1)
            try:
                mesh = efish_cloudvolume.mesh.get(int(id))
                vertices = list(mesh.values())[0].vertices
                vertices = vertices/scale_factor
                vertices = [[v[0]-refpt[0],v[1]-refpt[1],v[2]-refpt[2]] for i,v in enumerate(vertices)]
                vertices = [[v[0],yaxis_replace_scale-v[1],v[2]] for v in vertices]
                faces = list(mesh.values())[0].faces
                trimesh_mesh = trimesh.Trimesh(vertices=vertices, faces=faces,process=True)
                
                # downsample the fused mesh before joining to the fused_mesh and exporting as a .obj for blender
                simplified_mesh = trimesh_mesh.simplify_quadric_decimation(face_count=len(trimesh_mesh.vertices) * reduction_ratio)
                fused_mesh = trimesh.util.concatenate([fused_mesh, simplified_mesh])

            except MeshDecodeError as e: 
                print(f"An unexpected error occurred on segment{id}: {e}")
                continue

            except ValueError as e:
                print(f"An unexpected error occurred on segment{id}: {e}")
                continue
    
        # prompt: downsample the fused mesh before exporting as a .obj for blender
        # reduction_ratio = 0.2
        # simplified_mesh = fused_mesh.simplify_quadric_decimation(face_count=len(fused_mesh.vertices) * reduction_ratio)


    fused_mesh.export(savepath / f'{ctype}_{focal_cell_id}_axon.obj');

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 174/174 [01:25<00:00,  2.03it/s]


# OBJ from Local Mesh CloudVolume retrieve http

In [48]:
vol = CloudVolume('precomputed://http://localhost:1337')

In [49]:
segIDs_all = vol.unique(bbox=vol.bounds) - set([0]) # get all segment IDs in volume and remove segment ID 0, which is background

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 572/572 [00:04<00:00, 122.06it/s]


In [50]:
savepath = Path('/Users/kperks/Library/CloudStorage/GoogleDrive-sawtelllab@gmail.com/My Drive/ELL_connectome/ms/figures_final/blender/obj/Fig1/OUTPUT_in-sub-vol')

for s in list(segIDs_all):
        
    mesh= vol.mesh.get(int(s))
    
    # reduction_ratio = 0.2
    
    scale_factor = 100000
    yaxis_replace_scale = 32768*16/scale_factor
    # zaxis_replace_scale = 3534*30/scale_factor
    refpt = [0,0,0]
    
    vertices = mesh.vertices
    vertices = vertices/scale_factor
    vertices = [[v[0]-refpt[0],v[1]-refpt[1],v[2]-refpt[2]] for i,v in enumerate(vertices)]
    vertices = [[v[0],yaxis_replace_scale-v[1],v[2]] for v in vertices]
    faces = mesh.faces
    trimesh_mesh = trimesh.Trimesh(vertices=vertices, faces=faces,process=True)

    # downsample the fused mesh before joining to the fused_mesh and exporting as a .obj for blender
    # simplified_mesh = trimesh_mesh.simplify_quadric_decimation(face_count=len(trimesh_mesh.vertices) * reduction_ratio)
    
    
    trimesh_mesh.export(savepath / f'{s}.obj')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:01<00:00, 18.06it/s]
